In [1]:
import pandas as pd

data = pd.read_csv('partial_results.csv')

In [2]:
data

,ST_ID,LATITUDE,LONGITUDE,NAME,OSM_ID
0,2.0,48.4272,42.2162,Чернышков,1.401090e+09
1,6.0,48.5423,42.5028,Обливская,1.398621e+09
2,8.0,48.6062,42.8445,Суровикино,3.745996e+09
3,12.0,48.4829,43.1614,Чир,8.713151e+09
4,15.0,48.4400,43.3798,Ложки,8.713142e+09
...,...,...,...,...,...
15169,25877.0,56.6841,57.9468,NaN,NaN
15170,25878.0,56.8725,59.2641,NaN,NaN
15171,25879.0,56.4072,52.9355,NaN,NaN
15172,25880.0,56.2014,52.6143,NaN,NaN


In [8]:
import numpy as np
data[data['OSM_ID'].notna()]

,ST_ID,LATITUDE,LONGITUDE,NAME,OSM_ID
0,2.0,48.4272,42.2162,Чернышков,1.401090e+09
1,6.0,48.5423,42.5028,Обливская,1.398621e+09
2,8.0,48.6062,42.8445,Суровикино,3.745996e+09
3,12.0,48.4829,43.1614,Чир,8.713151e+09
4,15.0,48.4400,43.3798,Ложки,8.713142e+09
...,...,...,...,...,...
15161,25869.0,55.6896,37.6170,Нагатинская,2.969531e+08
15162,25870.0,55.7207,37.5608,Спортивная,2.786931e+08
15163,25871.0,55.7478,37.5319,Кутузовская,2.411584e+08
15164,25872.0,55.7891,37.5041,Октябрьское Поле,3.097489e+08


In [11]:
import pyrosm
import geopandas as gpd
from shapely.geometry import Point


def get_moscow_region_boundaries():
    osm = pyrosm.OSM("data/moscow_region.pbf")
    moscow_region = osm.get_boundaries("Московская область")

    return moscow_region

def check_points_in_moscow_region(points, moscow_region):
    geometry = [Point(xy) for xy in zip(points['Longitude'], points['Latitude'])]
    geo_df = gpd.GeoDataFrame(points, geometry=geometry, crs="EPSG:4326")

    points_in_region = []
    for point in geo_df.itertuples():
        point_geom = Point(point.geometry)
        for poly in moscow_region.itertuples():
            if point_geom.within(poly.geometry):
                points_in_region.append(True)
                break
        else:
            points_in_region.append(False)

    geo_df['InMoscowRegion'] = points_in_region

    return geo_df[['Latitude', 'Longitude', 'InMoscowRegion']]

# Загрузка границ Московской области
moscow_region = get_moscow_region_boundaries()

ValueError: File does not exist: Found: data/moscow_region.pbf

In [16]:
result['InMoscowRegion'].value_counts()

InMoscowRegion
False    15174
Name: count, dtype: int64